In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings('ignore')

In [41]:
#data preprocessing
# from pandas.io.parsers.readers import IndexLabel

df = pd.read_csv("/Train.csv")
df['Outlet_Size'] = ['Unknown' if pd.isna(value) else value for value in df['Outlet_Size']]
df_weights = df['Item_Weight'].mean()
df['Item_Weight'].fillna(df_weights, inplace=True)
df= df.drop(labels=['Item_Identifier','Outlet_Identifier','Item_Type'] ,axis = 1 )

''' in above i used to drop item type but i canceled it , it may be essential in prediction   '''

#categorical 2 numerical

df['Item_Fat_Content']=['Low Fat'if item =="LF" else item for item in df['Item_Fat_Content']]
df['Item_Fat_Content']=['Low Fat'if item =="low fat" else item for item in df['Item_Fat_Content']]
df['Item_Fat_Content']=['Regular'if item =="reg" else item for item in df['Item_Fat_Content']]

new_df = pd.get_dummies(df)

#  sending item outlet sales into the end of dataset

# Replace 'column_name' with the name of the column you want to move
column_name = 'Item_Outlet_Sales'

# Pop the column from its current position
column_to_move = new_df.pop(column_name)

# Add the popped column back to the DataFrame at the end
new_df[column_name] = column_to_move


header_names = new_df.columns
new_df




,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Fat_Content_Low Fat,Item_Fat_Content_Regular,Outlet_Size_High,Outlet_Size_Medium,Outlet_Size_Small,Outlet_Size_Unknown,Outlet_Location_Type_Tier 1,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 3,Outlet_Type_Grocery Store,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3,Item_Outlet_Sales
0,9.300,0.016047,249.8092,1999,1,0,0,1,0,0,1,0,0,0,1,0,0,3735.1380
1,5.920,0.019278,48.2692,2009,0,1,0,1,0,0,0,0,1,0,0,1,0,443.4228
2,17.500,0.016760,141.6180,1999,1,0,0,1,0,0,1,0,0,0,1,0,0,2097.2700
3,19.200,0.000000,182.0950,1998,0,1,0,0,0,1,0,0,1,1,0,0,0,732.3800
4,8.930,0.000000,53.8614,1987,1,0,1,0,0,0,0,0,1,0,1,0,0,994.7052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8518,6.865,0.056783,214.5218,1987,1,0,1,0,0,0,0,0,1,0,1,0,0,2778.3834
8519,8.380,0.046982,108.1570,2002,0,1,0,0,0,1,0,1,0,0,1,0,0,549.2850
8520,10.600,0.035186,85.1224,2004,1,0,0,0,1,0,0,1,0,0,1,0,0,1193.1136
8521,7.210,0.145221,103.1332,2009,0,1,0,1,0,0,0,0,1,0,0,1,0,1845.5976


In [47]:
#train test split
#changed new_df to df
X = new_df.iloc[:,:-1]
y = new_df.iloc[:,-1]
# X_train , X_test , y_train , y_test = train_test_split(X, y , test_size=0.2 , random_state = 42)
# X_train.shape

In [ ]:
#LinearRegression training phase

lr = LinearRegression()
lr.fit(X_train , y_train)
# print("the intercept",lr.intercept_)
# print("\n the coefs = ",lr.coef_)

y_pred = lr.predict(X_test)


In [ ]:
#LinearRegression evaluation phase

r2score = r2_score( y_test , y_pred)
print ('r2 score is : ',r2score)

rmse = np.sqrt(mean_squared_error(y_test , y_pred))
print ('RMSE is = ',rmse)


r2 score is :  0.5801042361385736
RMSE is =  1068.2996383386642


In [49]:
#Random forest training
rf = RandomForestRegressor(n_estimators=100, random_state=42)


In [50]:
#cross validation

cv_scores = cross_val_score(rf, X, y, cv=5, scoring='neg_mean_squared_error')
cv_rmse_scores = np.sqrt(-cv_scores)
# Print the RMSE scores for each fold
for fold, rmse in enumerate(cv_rmse_scores, start=1):
    print(f'Fold {fold}: RMSE = {rmse:.2f}')

# Calculate the mean RMSE score across all folds
mean_rmse = np.mean(cv_rmse_scores)
print(f'Mean RMSE across folds: {mean_rmse:.2f}')

Fold 1: RMSE = 1149.90
Fold 2: RMSE = 1140.90
Fold 3: RMSE = 1133.34
Fold 4: RMSE = 1135.64
Fold 5: RMSE = 1135.10
Mean RMSE across folds: 1138.98


In [ ]:
#random forest evaluation phase

r2score = r2_score( y_test , rfy_pred)
print ('r2 score is : ',r2score)

rmse = np.sqrt(mean_squared_error(y_test , rfy_pred))
print ('RMSE is = ',rmse)

r2 score is :  0.5517707482543488
RMSE is =  1103.7543609094785


In [ ]:
# Create a Gradient Boosting Regressor object
gb_regressor = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42)

# Fit the model to the training data
gb_regressor.fit(X_train, y_train)

# Predict on the test data
y_pred_gb = gb_regressor.predict(X_test)

# Calculate metrics like R-squared and RMSE
r2_gb = r2_score(y_test, y_pred_gb)
rmse_gb = np.sqrt(mean_squared_error(y_test, y_pred_gb))

print('Gradient Boosting R2 score:', r2_gb)
print('Gradient Boosting RMSE:', rmse_gb)


Gradient Boosting R2 score: 0.6018397063436576
Gradient Boosting RMSE: 1040.282538483302
